In [ ]:
# Hacky way to schedule. Here I'm setting these to sleep until the gpus should be free.
# At the end of the notebooks  os._exit(00) will kill the kernel freeing the gpu. 
#                          Hours to wait
# import time; time.sleep( 6 * (60*60))

# G only ACGT Probabilities Conv1D  

> 

In [ ]:
import os, json, re

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

import hilbertcurve
from hilbertcurve.hilbertcurve import HilbertCurve

from EnvDL.core import * # includes remove_matching_files
from EnvDL.dna  import *
from EnvDL.dlfn import * # includes LSUV_

from tqdm import tqdm

# import os
# import numpy as np
# import pandas as pd

# import plotly.express as px
# import plotly.io as pio
# pio.templates.default = "plotly_white"

# import hilbertcurve
# from hilbertcurve.hilbertcurve import HilbertCurve

# from EnvDL.core import * # includes remove_matching_files
# from EnvDL.dna import *
# from EnvDL.dlfn import *

# from tqdm import tqdm

In [ ]:
use_gpu_num = 0

# Imports --------------------------------------------------------------------
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F # F.mse_loss

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger


device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [ ]:
cache_path = '../nbs_artifacts/02.22_g2fc_G_ACGT_conv1d/'
ensure_dir_path_exists(dir_path = cache_path)

## Load data

In [ ]:
load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
# phno = pd.read_csv(load_from+'phno.csv')
phno_geno = pd.read_csv(load_from+'phno_geno.csv')
phno = phno_geno

obs_geno_lookup = np.load(load_from+'obs_geno_lookup.npy') # Phno_Idx	Geno_Idx	Is_Phno_Idx
YMat = np.load(load_from+'YMat.npy')
# GMat = np.load(load_from+'GMat.npy')
# ACGT_OneHot = np.load(load_from+'ACGT_OneHot.npy')
ACGT = np.load(load_from+'ACGT.npy')
# ACGT_hilb = np.load(load_from+'ACGT_hilb.npy')
# SMat = np.load(load_from+'SMat3.npy')
# WMat = np.load(load_from+'WMat3.npy')
# MMat = np.load(load_from+'MMat3.npy')

In [ ]:
# reduce Size of ACGT ----
# Since these are probabilities we can immediately save 1/4 of the space needed by using ACG instead of ACGT
# ACGT = ACGT[:, 0:-1, :]

### Create train/test validate indicies from json

In [ ]:
load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/'

split_info = read_split_info(
    load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/',
    json_prefix = '2023:9:5:12:8:26')

temp = phno.copy()
temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)

test_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['test'][0]
)

temp = temp.loc[test_dict['train_idx'], ] # restrict before re-aplying

val_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['validate'][0]
)

In [ ]:
train_idx = val_dict['train_idx']
test_idx  = val_dict['test_idx']

In [ ]:
# confirm all observation idxs are have genomic information
assert [] == [e for e in list(train_idx)+list(test_idx) if e not in obs_geno_lookup[:, 0]]

## One Hot Encoded

In [ ]:
YMat_cs = calc_cs(YMat[train_idx])
y_cs = apply_cs(YMat, YMat_cs)

In [ ]:
training_dataloader = DataLoader(
    ACGTDataset(y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float), 
                G = torch.from_numpy(ACGT).to(torch.float), 
                idx_original = torch.from_numpy(np.array(train_idx)),
                idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
                use_gpu_num = 0,
                device = 'cuda'
               ),
    batch_size = 50,
    shuffle = True
)

In [ ]:
validation_dataloader = DataLoader(
    ACGTDataset(y = torch.from_numpy(y_cs[test_idx])[:, None].to(torch.float), 
                G = torch.from_numpy(ACGT).to(torch.float), 
                idx_original = torch.from_numpy(np.array(test_idx)),
                idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
                use_gpu_num = 0,
                device = 'cuda'
               ),
    batch_size = 50,
    shuffle = True
)

In [ ]:
next(iter(training_dataloader))[0].shape

torch.Size([50, 4, 125891])

In [ ]:
# Using https://pytorch.org/vision/main/_modules/torchvision/models/resnet.html as a starting point

def conv0x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv1d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


In [ ]:

def conv0x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv1d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [ ]:
conv0x3(4, 4, 2)(torch.randn(50, 4, 125891)).shape

torch.Size([50, 4, 62946])

In [ ]:
class BasicBlock1d(nn.Module):
    expansion = 1
    def __init__(self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: nn.Module = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: nn.Module = None,
        expansion: int = 1
    ) -> None:
        super().__init__()
        # Set up defaults if none was passed in
        if norm_layer is None:
            norm_layer = nn.BatchNorm1d
        if groups != 1 or base_width !=  64: raise ValueError('Only groups = 1 and base_width = 64 supported')
        if dilation > 1: raise NotImplementedError('Dilation not supported')

        # self.expansion = expansion # in ResNet v1, this is 1, in ResNet v1.5 it is set to 4

        # self.conv1 = conv3x3(inplanes, planes*self.expansion, stride)
        self.conv1 = conv0x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        # self.conv2 = conv3x3(planes, planes*self.expansion)
        self.conv2 = conv0x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
  

In [ ]:
  
class BottleneckBlock1d(nn.Module):
    # This is the block used in ResNet v1.5. It is supposed to be more effective.
    # Main changes are that 
    # - expansion is not set to 1 
    # - now there is a third convolution that happens in the slow path
    #
    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: nn.Module = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: nn.Module = None,
        expansion: int = 4
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm1d
            
        self.expansion = expansion

        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv0x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv0x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv0x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
    


In [ ]:


class ResNet1d(nn.Module):
    def __init__(
        self,
        block, #: Type[Union[BasicBlock, Bottleneck]],
        layers, #: List[int],
        num_outputs: int = 1,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation = None, #: Optional[List[bool]] = None,
        norm_layer = None, #: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        # _log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm1d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv1d(4, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False) # Note that this is 4 not 3
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool1d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_outputs)

        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm1d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block, #: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv0x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: torch.Tensor) -> torch.Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self._forward_impl(x)




In [ ]:
def Conv1D_Max_block(in_channels, out_channels, kernel_size, stride):
    block = nn.Sequential(
        nn.Conv1d(
            in_channels= in_channels, # second channel
            out_channels= out_channels,
            kernel_size= kernel_size,
            stride= stride
        ), 
        nn.MaxPool1d((kernel_size,), stride=stride)
    )
    return(block)


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()    

#         def Linear_block(in_size, out_size, drop_pr):
#             block = nn.Sequential(
#                 nn.Linear(in_size, out_size),
#                 nn.ReLU(),
#                 nn.Dropout(drop_pr)
#             )
#             return(block)         
        
        
#         def Conv1D_Max_block(in_channels, out_channels, kernel_size, stride):
#             block = nn.Sequential(
#                 nn.Conv1d(
#                     in_channels= in_channels, # second channel
#                     out_channels= out_channels,
#                     kernel_size= kernel_size,
#                     stride= stride
#                 ), 
#                 nn.MaxPool1d((kernel_size,), stride=stride)
#             )
#             return(block)
        
        self.x_network = nn.Sequential(
            Conv1D_Max_block(3, 3, 3, 2),
            Conv1D_Max_block(3, 3, 3, 2),
            Conv1D_Max_block(3, 3, 3, 2),
            Conv1D_Max_block(3, 3, 3, 2),
            Conv1D_Max_block(3, 3, 3, 2),
            Conv1D_Max_block(3, 3, 3, 2)
        )
        
    
        self.x_pred = nn.Sequential(
                nn.Flatten(),           
                Linear_block(in_size = 87, out_size = 32, drop_pr = 0.3),
                nn.Linear(32, 1)
            )

    def forward(self, x):
        out = self.x_network(x)
        pred = self.x_pred(out)
        return pred, out

#     def forward(self, x):
#         x_out = self.x_network(x)
#         return x_out

model = NeuralNetwork().to(device)

# model(next(iter(training_dataloader))[0][0:5])[0].shape

In [ ]:
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()    

# #         def Linear_block(in_size, out_size, drop_pr):
# #             block = nn.Sequential(
# #                 nn.Linear(in_size, out_size),
# #                 nn.ReLU(),
# #                 nn.Dropout(drop_pr)
# #             )
# #             return(block)         
        
        
# #         def Conv1D_Max_block(in_channels, out_channels, kernel_size, stride):
# #             block = nn.Sequential(
# #                 nn.Conv1d(
# #                     in_channels= in_channels, # second channel
# #                     out_channels= out_channels,
# #                     kernel_size= kernel_size,
# #                     stride= stride
# #                 ), 
# #                 nn.MaxPool1d((kernel_size,), stride=stride)
# #             )
# #             return(block)
        
#         self.x_network = nn.Sequential(
#             nn.Conv2d(
#                     in_channels= 4, 
#                     out_channels= 4,
#                     kernel_size= (3, 3),
#                     stride= 2,
#                     padding = 1,
#                     bias = True
#                 ),
#             nn.Conv2d(
#                     in_channels= 4, 
#                     out_channels= 4,
#                     kernel_size= (3, 3),
#                     stride= 2,
#                     padding = 1,
#                     bias = True
#                 ),
#             nn.Conv2d(
#                     in_channels= 4, 
#                     out_channels= 4,
#                     kernel_size= (3, 3),
#                     stride= 2,
#                     padding = 1,
#                     bias = True
#                 ),
#             nn.Conv2d(
#                     in_channels= 4, 
#                     out_channels= 4,
#                     kernel_size= (3, 3),
#                     stride= 2,
#                     padding = 1,
#                     bias = True
#                 ),
#             nn.Conv2d(
#                     in_channels= 4, 
#                     out_channels= 4,
#                     kernel_size= (3, 3),
#                     stride= 2,
#                     padding = 1,
#                     bias = True
#                 )
#         )
        
#         self.x_pred = nn.Sequential(
#             nn.Flatten(),            
#             nn.Linear(512, 1)
#         )
        
#     def forward(self, x):
#         out = self.x_network(x)
#         pred = self.x_pred(out)
#         return pred, out

# model = NeuralNetwork().to(device)

# # model(next(iter(training_dataloader))[0])[0].shape

# # torch.Size([50, 4, 256, 512])

In [ ]:
LSUV_(model, next(iter(training_dataloader))[0])

In [ ]:
# Module for training subnetworks.
class plDNN_ACGT(pl.LightningModule):
    def __init__(self, mod):
        super().__init__()
        self.mod = mod
        
    def training_step(self, batch, batch_idx):
        g_i, y_i = batch
        pred, out = self.mod(g_i)
        loss = F.mse_loss(pred, y_i)
        self.log("train_loss", loss)
        
        with torch.no_grad():
            weight_list=[(name, param) for name, param in model.named_parameters() if name.split('.')[-1] == 'weight']
            for l in weight_list:
                self.log(("train_mean"+l[0]), l[1].mean())
                self.log(("train_std"+l[0]), l[1].std())        
        return(loss)
        
    def validation_step(self, batch, batch_idx):
        g_i, y_i = batch
        pred, out = self.mod(g_i)
        loss = F.mse_loss(pred, y_i)
        self.log('val_loss', loss)        
     
    def configure_optimizers(self, **kwargs):
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer    

In [ ]:
max_epoch = 200
DNNG = plDNN_ACGT(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-conv1")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

In [ ]:
torch.save(DNNG.mod, cache_path+'g-acgt-conv1'+'.pt')